# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [18]:
# Check the call column summary
data.call.describe()

count    4870.000000
mean        0.080493
std         0.272083
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In [5]:
# Count the number of resumes for each group
data.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

The sample size is 2435 for each group.

In [27]:
# Count the total callbacks
len(data[data.call==1])

392

In [10]:
# Count callbacks for each group 
data[data.call==1].race.value_counts()

w    235
b    157
Name: race, dtype: int64

In [46]:
# Count no callbacks for each group 
data[data.call==0].race.value_counts()

b    2278
w    2200
Name: race, dtype: int64

In [47]:
# Overall sample proportion of callbacks
p_hat = len(data[data.call==1])/len(data)
round(p_hat,4)

0.0805

In [32]:
# Sample proportion of callbacks for each group (p1_hat, p2_hat)
data[data.call==1].race.value_counts()/data.race.value_counts()

w    0.096509
b    0.064476
Name: race, dtype: float64

In [57]:
def print_result(my_tuple, title, label_1='test statistic', 
                 label_2='p-value', digits = 4):
    print(title)
    print(label_1, ": ", round(my_tuple[0],digits))
    print(label_2, ": ", round(my_tuple[1],digits))

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### Q1. What test is appropriate for this problem? Does CLT apply?

The goal of this study is finding out if race has a significant impact on the rate of callbacks for resumes. Thus, I will test if the callback rates for white-sounding resumes are significantly differeent from those for black-sounding resumes. The appropriate test for comparing the proportions of two groups is the __two proportion z-test__. 

The samples are random and independent within each group and less than 10% of the population. Moreover, the sample size is big enough since np and n(1-p) are 235 and 2200 for white-sounding names and 157 and 2278 for black-sounding names (all are much bigger than 10!!). Thus, the Central Limit Theorem can be applied to assume the proportion distributions are approximately normal.

### Q2. What are the null and alternate hypotheses?

- Null hypothesis: the population proportions of callbacks for the two groups (white-sounding vs. black-sounding) are the same i.e. _p1_ = _p2_
- Alternative hypothesis: the population proportions of callbacks for the two groups are different i.e. _p1_ != _p2_ (two-sided)

### Q3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [94]:
data_w = data[data.race=='w']
data_b = data[data.race=='b']

#### Frequentist approach

In [52]:
# Count callbacks for each group 
data[data.call==1].race.value_counts()

w    235
b    157
Name: race, dtype: int64

In [53]:
# Count the number of resumes for each group
data.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [70]:
count = np.array([235, 157]) # white vs. black callbacks
total = np.array([2435, 2435]) # white vs. black total counts

_Two-sample proportion z-test_

In [114]:
# two-sample proportion z-test (two-sided)
print_result(proportions_ztest(count,total), 
             "two-sample proportion z-test (two-sided)", digits=10)

two-sample proportion z-test (two-sided)
test statistic :  4.1084121524
p-value :  3.98389e-05


I used the two-sample z-test for proportions and found the p-value is close to zero. Thus, the null hypothesis can be rejected; the callback rate of white-sounding name resumes is significantly different from the callback rate of black-sounding name resumes. A resume with a white-sounding name is more likely to get a callback than that with a black-sounding name.

_Confidence interval for the difference of two population proportions_

In [85]:
def conf_interval_2sample_prop(counts, totals, alpha, digits):
    '''
    Input
    counts: an aray of two integers,number of successes in each group
    totals: an aray of two integers, number of total counts in each group
    alpha: significance level (1-confidence level)
    digits: number of decimal points to be printed
    Output (by printing)
    sample proportions, critical value, margin of error, 
    confidence interval for the difference of two population proportions
    '''
    # sample proportions
    n1, n2 = totals
    p1_hat = counts[0]/n1
    p2_hat = counts[1]/n2
    print("sample proportion of group 1:", round(p1_hat, digits))
    print("sample proportion of group 2:", round(p2_hat, digits))
    # critial value for given alpha
    CV = stats.norm.ppf(1-alpha/2) #critical value
    #print("critical value:", round(CV, digits))
    # standard error
    SE = np.sqrt((p1_hat*(1-p1_hat)/n1)+(p2_hat*(1-p2_hat)/n2))
    # margin of error
    MOE = CV*SE
    print("Margin of Error:", round(MOE, digits))
    # conf. interval
    conf_int = [round(p1_hat-p2_hat - MOE,digits), round(p1_hat-p2_hat + MOE,digits)]
    print(int((1-alpha)*100), "%", "conf. interval for prop. difference:", conf_int)

In [91]:
# conf. interval for the difference in population proportions
conf_interval_2sample_prop(count, total, 0.05, digits=5)

sample proportion of group 1: 0.09651
sample proportion of group 2: 0.06448
Margin of Error: 0.01526
95 % conf. interval for prop. difference: [0.01678, 0.04729]


The confidence interval for the difference in two population proportions of callbacks does not contain zero. This means the callback rate for a white-sounding resume is significantly higher than that for a black-sounding resume. This result is consistent with the hypothesis test result.

#### Bootstrapping approach

_Two-sample bootstrap hypothesis test for the proportion difference_

In [93]:
# Compute the proportion of callbacks for all observations
mean_prop = np.mean(data.call)
mean_prop

0.08049282

In [96]:
# Compute the proportion of callbacks for each group
call_w = data_w.call
call_b = data_b.call
print(np.mean(call_w), np.mean(call_b))

0.09650924 0.064476386


In [106]:
# difference in sample proportions
empirical_diff = np.mean(call_w) - np.mean(call_b)
empirical_diff

0.032032855

In [109]:
# Take bootstrap replicates of call values 
def draw_bs_replicates(data, group_size, func, size):
    return np.array([func(np.random.choice(data, group_size)) for _ in range(size)])

# Compute 10,000 bootstrap replicates
bs_replicates_w = draw_bs_replicates(data.call, len(call_w), np.mean, 10000) 
bs_replicates_b = draw_bs_replicates(data.call, len(call_b), np.mean, 10000)

# Get replicates of difference of proportions
bs_replicates_diff = bs_replicates_w - bs_replicates_b

# Compute and print p-value: p
p = np.sum(bs_replicates_diff >  empirical_diff ) / len(bs_replicates_diff)
print('p-value =', p)

p-value = 0.0


_Boostrap Confidence interval for the difference in proportions_

In [112]:
# Bootstrap confidence interval
bs_replicates_w = draw_bs_replicates(call_w, len(call_w), np.mean, 10000) 
bs_replicates_b = draw_bs_replicates(call_b, len(call_b), np.mean, 10000)

# Get replicates of difference of proportions
bs_replicates_diff = bs_replicates_w - bs_replicates_b
conf_int = np.percentile(bs_replicates_diff, [2.5, 97.5])
print("95% Bootstrap conf. interval of difference in proportions: ", conf_int)

95% Bootstrap conf. interval of difference in proportions:  [0.01642711 0.04722793]


Bootstrap methods gave a p-value (close to zero) and a confidence interval similar to those of frequentist methods. Thus, the conclusion is the same as above.

### Q4. Write a story describing the statistical significance in the context or the original problem.


We found the significant difference between the callback rates for white-sounding name resumes and black-sounding ones. The direction of the difference tells us that the callback rates for white-sounding name resumes is significantly higher. This shows that a resume with a white-sounding name is more likely to get a callback for interviews from employers than a resume with a black-sounding name. Is there really racial discrimination going on in the US job market?

### Q5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

My analysis does NOT mean the race/name is the most important factor in callback success since I did not check other factors in the dataset. There might have been a more crucial factor in the dataset (or even outside) than race/name. I will investigate other factors and describe how my analysis can be amended.

I will first check whether resumes were fairly distributed into two groups.

In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

There are a total of 65 columns, 55 numeric and 10 non-numeric. I will first check the numeric columns using mean.

In [119]:
data.groupby('race').mean().transpose()

race,b,w
education,3.616016,3.620945
ofjobs,3.658316,3.664476
yearsexp,7.829569,7.856263
honors,0.051335,0.054209
volunteer,0.414374,0.408624
military,0.101848,0.092402
empholes,0.445996,0.450103
occupspecific,216.744969,214.530595
occupbroad,3.487885,3.475154
workinschool,0.560986,0.558111


The above shows the means for white-sounding names and black-sounding names for each column. The means are very similar for the two groups except for the means for callbacks. It looks like the resumes were almost identically assigned to white-sounding and black-sounding names. I found no problem here. Now I will check the non-numeric columns.

In [165]:
# string columns
str_cols = [data.columns[i] for i, value in enumerate(data.iloc[0,:]) if type(value) is str]
print(str_cols)

['id', 'ad', 'firstname', 'sex', 'race', 'city', 'kind', 'expminreq', 'schoolreq', 'ownership']


These are the ten non-numeric columns.

In [149]:
data[str_cols].head(10)

,id,ad,firstname,sex,race,city,kind,expminreq,schoolreq,ownership
0,b,1,Allison,f,w,c,a,5,,
1,b,1,Kristen,f,w,c,a,5,,
2,b,1,Lakisha,f,b,c,a,5,,
3,b,1,Latonya,f,b,c,a,5,,
4,b,1,Carrie,f,w,c,a,some,,Nonprofit
5,b,1,Jay,m,w,c,s,,,Private
6,b,1,Jill,f,w,c,s,,,Private
7,b,1,Kenya,f,b,c,a,some,,Nonprofit
8,b,1,Latonya,f,b,c,s,,,Private
9,b,1,Tyrone,m,b,c,s,,,Private


In [173]:
data[str_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 10 columns):
id           4870 non-null object
ad           4870 non-null object
firstname    4870 non-null object
sex          4870 non-null object
race         4870 non-null object
city         4870 non-null object
kind         4870 non-null object
expminreq    4870 non-null object
schoolreq    4870 non-null object
ownership    4870 non-null object
dtypes: object(10)
memory usage: 578.5+ KB


In [182]:
data.expminreq[5]==''

True

In [186]:
# number of missing values for each string column
for col in str_cols:
    print(col, len([1 for val in data[col] if val=='']))

id 0
ad 0
firstname 0
sex 0
race 0
city 0
kind 0
expminreq 2746
schoolreq 4350
ownership 1992


There are some missing values with empty strings which cannot be caught in the summary from info(). I will be careful with the columns with too many missing values and find the number of distinct values for each string column.

In [202]:
# number of distinct values for each string column
for col in str_cols:
    print(col, len(data[data.call==1][col].value_counts()/data[col].value_counts()))

id 289
ad 303
firstname 36
sex 2
race 2
city 2
kind 2
expminreq 13
schoolreq 4
ownership 4


Only sex, race, city, and kind, ownership columns have manageable numbers of disctinct values (schoolreq was not considered since it has 4350 missing values). I will calculate the callback rate for each level of these columns

In [203]:
for col in ['sex','race','city','kind','ownership']:
    print(data[data.call==1][col].value_counts()/data[col].value_counts())
    print()

f    0.082488
m    0.073843
Name: sex, dtype: float64

w    0.096509
b    0.064476
Name: race, dtype: float64

b    0.096953
c    0.067308
Name: city, dtype: float64

a    0.084885
s    0.074591
Name: kind, dtype: float64

             0.088855
Nonprofit    0.053459
Private      0.075914
Public       0.084507
Name: ownership, dtype: float64



All of these columns have pretty different callback rates for different values. In particular, 'city' column has a big difference in callback rates between two groups 'b' and 'c' and the difference is almost as big as the difference for the 'race' column. The ownership column is also interesting. The callback rate is the highest for missing values and it decreases in the order of public, private, and nonprofit. There are these other factors that have effects on the callback rates, so we cannot conclude that the race/name is the most important factor. 

How can we amend the analysis?

If these 2435 resumes for black-sounding names and 2435 resumes for white-sounding names were exactly the same (using means I found they are not exactly the same) and two observations of white and black names can be paired with the same resume, the paired z-test for proportion could be more appropriate to rule out effects of other factors. If they cannot be paired, we can also try multivariate analysis. A logistic regression can be used to predict a callback (binary output, 0 or 1) and find all significant factors.

__End of answers!!!__ 

_Please ignore below_

In [166]:
#data.groupby('race').firstname.value_counts()

In [167]:
#len(data.groupby('race').firstname.value_counts())

In [168]:
#These are the black-sounding and white-sounding names used. The name pool was smaller than I expected. We can also check if there is any name that leads to more callbacks.

In [169]:
#data['sex'].value_counts()

In [170]:
#There were more female names than male ones. This is okay as long as each gender was evenly distributed to the two race/name groups.

In [171]:
# Proportions of females 
#data[data.sex=='f'].race.value_counts()/data.race.value_counts()

In [ ]:
# There are slightly more female names in black-sounding names, but they look fine.